In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess, time, os, requests, shutil, zipfile, json, re
from PIL import Image
from google.colab import files

# Start Ollama for any future LLM-based logic
subprocess.Popen(["ollama", "serve"])
time.sleep(5)
!ollama pull qwen2.5:0.5b
print("[INFO] Environment initialization complete")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

✅ Environment Ready


In [ ]:
import os, shutil, zipfile, json, re, requests
from PIL import Image
from google.colab import files

class BalancedItemAgent:
    def __init__(self, model="qwen2.5:0.5b"):
        self.url = "http://localhost:11434/api/generate"
        self.model = model
        self.work_dir = "/content/agent_workspace"
        self.project_dir = os.path.join(self.work_dir, "mod_project")
        self.mod_id = ""
        self.package_path = ""
        self.main_class_name = ""
        os.makedirs(self.work_dir, exist_ok=True)

    def setup_project(self, zip_path):
        if os.path.exists(self.project_dir): shutil.rmtree(self.project_dir)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(self.project_dir)
        fabric_json = os.path.join(self.project_dir, "src/main/resources/fabric.mod.json")
        with open(fabric_json, 'r') as f:
            data = json.load(f)
            self.mod_id = data.get('id', 'modid')
            entrypoint = data['entrypoints']['main'][0]
            self.main_class_name = entrypoint.split('.')[-1]
            self.package_path = entrypoint.rsplit('.', 1)[0].replace('.', '/')
        print(f"[SUCCESS] Mod loaded: {self.mod_id} | Main class: {self.main_class_name}")

    def get_balanced_logic(self, description):
        prompt = f"""[INST] You are a Minecraft Mod Data Bot. Analyze the user request and determine what to create.

USER REQUEST: {description}

DECISION RULES (apply in order):
1. If user asks for something CRAFTABLE/EDIBLE → TYPE: FOOD
2. Else if user asks for something to BREW in brewing stand → TYPE: POTION
3. Else if user asks for a custom STATUS EFFECT (something entities can have applied to them) → TYPE: EFFECT
4. Else → TYPE: FOOD (default)

VANILLA EFFECTS LIST (only use these, don't create custom unless effect not in list):
speed, slowness, haste, mining_fatigue, strength, instant_health, instant_damage, jump_boost, nausea, regeneration, resistance, fire_resistance, water_breathing, invisibility, blindness, night_vision, hunger, weakness, poison, wither, health_boost, absorption, saturation, glowing, levitation, luck, unluck, slow_falling, conduit_power, dolphins_grace, bad_omen, hero_of_the_village

OUTPUT FORMAT (ONE TYPE ONLY - do not output multiple types):

TYPE: [FOOD|POTION|EFFECT]
NEEDS_CUSTOM_EFFECT: [true|false]
CUSTOM_EFFECT_NAME: [name_if_needed or "none"]
CUSTOM_EFFECT_DESCRIPTION: [behavior_if_needed or "none"]

For FOOD:
nutrition=X, saturation=X.X, effect_id=MobEffects.VANILLA, duration_seconds=X, recipe_ingredients=[minecraft:item1, minecraft:item2], recipe_count=X

For POTION:
effect_id=MobEffects.VANILLA, duration_seconds=X, brewing_ingredient=minecraft:item, input_potion=water

For EFFECT:
effect_id=custom_name, category=[BENEFICIAL|HARMFUL|NEUTRAL], color_rgb=0xRRGGBB, effect_behavior=description

CRITICAL RULES:
- Output ONLY ONE TYPE line. Never output multiple TYPEs.
- recipe_ingredients MUST be [minecraft:item1, minecraft:item2] format
- For FOOD: nutrition 1-12, saturation 0.1-1.2
- For POTION: duration 20-3600 seconds
- For EFFECT: duration in seconds, describe tick behavior

[/INST]"""

        try:
            response = requests.post(self.url, json={"model": self.model, "prompt": prompt, "stream": False}, timeout=30)
            raw_text = response.json().get('response', '')

            print(f"[DEBUG] Raw AI response received\n")

            # Parse the type
            item_type = self._extract_field(raw_text, r'TYPE:\s*([A-Z]+)')
            needs_custom = self._extract_field(raw_text, r'NEEDS_CUSTOM_EFFECT:\s*(true|false)', default='false')
            custom_effect_name = self._extract_field(raw_text, r'CUSTOM_EFFECT_NAME:\s*([a-z_]+|none)')
            custom_effect_desc = self._extract_field(raw_text, r'CUSTOM_EFFECT_DESCRIPTION:\s*(.+?)(?=\n|$)')

            logic = {
                "type": item_type if item_type else "FOOD",
                "needs_custom_effect": needs_custom.lower() == 'true',
                "custom_effect_name": custom_effect_name if custom_effect_name != 'none' else None,
                "custom_effect_description": custom_effect_desc if custom_effect_desc != 'none' else None
            }

            # Parse based on type
            if logic["type"] == "FOOD":
                logic.update(self._parse_food_logic(raw_text))
            elif logic["type"] == "POTION":
                logic.update(self._parse_potion_logic(raw_text))
            elif logic["type"] == "EFFECT":
                logic.update(self._parse_effect_logic(raw_text))

            return logic

        except Exception as e:
            print(f"[ERROR] Parser error: {e}")
            return {"type": "FOOD", "nutrition": 6, "saturation": 0.6, "effects": [], "recipe_ingredients": ["minecraft:apple", "minecraft:sugar"], "recipe_count": 1}

    def _extract_field(self, text, pattern, default=None):
        """Helper to extract regex field"""
        match = re.search(pattern, text, re.IGNORECASE)
        return match.group(1) if match else default

    def _parse_food_logic(self, raw_text):
        """Parse food-specific fields"""
        nutrition = re.search(r'nutrition["\s:=]*([\d]+)', raw_text)
        saturation = re.search(r'saturation["\s:=]*([\d.]+)', raw_text)

        effect_field = re.search(r'effect_id["\s:=]*(.+?)(?=\s*(?:duration_seconds|recipe_|nutrition|saturation|$))', raw_text, re.IGNORECASE)
        effects = []
        if effect_field:
            effects = re.findall(r'MobEffects\.([A-Z_]+)|custom:([a-z_]+)', effect_field.group(1))

        duration_field = re.search(r'duration_seconds["\s:=]*(.+?)(?=\s*(?:recipe_|nutrition|saturation|$))', raw_text, re.IGNORECASE)
        durations = []
        if duration_field:
            durations = re.findall(r'(\d+)', duration_field.group(1))

        ingredients = re.findall(r'minecraft:[a-z_]+', raw_text)
        invalid_items = {
            'minecraft:speed', 'minecraft:slowness', 'minecraft:haste', 'minecraft:mining_fatigue',
            'minecraft:strength', 'minecraft:instant_health', 'minecraft:instant_damage',
            'minecraft:jump_boost', 'minecraft:nausea', 'minecraft:regeneration', 'minecraft:resistance',
            'minecraft:fire_resistance', 'minecraft:water_breathing', 'minecraft:invisibility',
            'minecraft:blindness', 'minecraft:night_vision', 'minecraft:hunger', 'minecraft:weakness',
            'minecraft:poison', 'minecraft:wither', 'minecraft:health_boost', 'minecraft:absorption',
            'minecraft:saturation', 'minecraft:glowing', 'minecraft:levitation', 'minecraft:luck',
            'minecraft:unluck', 'minecraft:slow_falling', 'minecraft:conduit_power', 'minecraft:dolphins_grace',
            'minecraft:bad_omen', 'minecraft:hero_of_the_village',
            'minecraft:crafting_shaped', 'minecraft:crafting_shapeless', 'minecraft:smelting',
            'minecraft:blasting', 'minecraft:smoking', 'minecraft:campfire_cooking', 'minecraft:stonecutting',
            'minecraft:smithing_transform', 'minecraft:smithing_trim'
        }
        ingredients = [ing for ing in ingredients if ing not in invalid_items]

        seen = set()
        deduped_ingredients = []
        for ing in ingredients:
            if ing not in seen:
                seen.add(ing)
                deduped_ingredients.append(ing)
        ingredients = deduped_ingredients

        if not ingredients or len(ingredients) < 2:
            ingredients = ["minecraft:apple", "minecraft:sugar"]

        effect_duration_pairs = []
        for i, effect_tuple in enumerate(effects):
            vanilla_effect = effect_tuple[0]
            custom_effect = effect_tuple[1]
            duration = int(durations[i]) if i < len(durations) else 10

            if vanilla_effect:
                effect_duration_pairs.append({
                    "effect_id": f"MobEffects.{vanilla_effect}",
                    "duration": duration * 20
                })
            elif custom_effect:
                effect_duration_pairs.append({
                    "effect_id": f"custom:{custom_effect}",
                    "duration": duration * 20
                })

        recipe_count = re.search(r'recipe_count["\s:=]*([1-8])', raw_text)

        return {
            "nutrition": max(1, min(int(nutrition.group(1)) if nutrition else 6, 12)),
            "saturation": max(0.1, min(float(saturation.group(1)) if saturation else 0.6, 1.2)),
            "effects": effect_duration_pairs,
            "recipe_ingredients": ingredients[:4],
            "recipe_count": max(1, min(int(recipe_count.group(1)) if recipe_count else 1, 8)),
            "has_effect": len(effect_duration_pairs) > 0
        }

    def _parse_potion_logic(self, raw_text):
        """Parse potion-specific fields"""
        effect_field = re.search(r'effect_id["\s:=]*(.+?)(?=\s*(?:duration_seconds|brewing|input|$))', raw_text, re.IGNORECASE)
        effects = []
        if effect_field:
            effects = re.findall(r'MobEffects\.([A-Z_]+)|custom:([a-z_]+)', effect_field.group(1))

        duration_field = re.search(r'duration_seconds["\s:=]*(.+?)(?=\s*(?:brewing|input|$))', raw_text, re.IGNORECASE)
        durations = []
        if duration_field:
            durations = re.findall(r'(\d+)', duration_field.group(1))

        brewing_ingredient = re.search(r'brewing_ingredient["\s:=]*(minecraft:[a-z_]+)', raw_text, re.IGNORECASE)
        input_potion = re.search(r'input_potion["\s:=]*([a-z_]+|minecraft:[a-z_]+)', raw_text, re.IGNORECASE)

        effect_duration_pairs = []
        for i, effect_tuple in enumerate(effects):
            vanilla_effect = effect_tuple[0]
            custom_effect = effect_tuple[1]
            duration = int(durations[i]) if i < len(durations) else 20

            if vanilla_effect:
                effect_duration_pairs.append({
                    "effect_id": f"MobEffects.{vanilla_effect}",
                    "duration": duration
                })
            elif custom_effect:
                effect_duration_pairs.append({
                    "effect_id": f"custom:{custom_effect}",
                    "duration": duration
                })

        return {
            "effects": effect_duration_pairs,
            "brewing_ingredient": brewing_ingredient.group(1) if brewing_ingredient else "minecraft:sugar",
            "input_potion": input_potion.group(1) if input_potion else "water",
            "has_effect": len(effect_duration_pairs) > 0
        }

    def _parse_effect_logic(self, raw_text):
        """Parse custom effect-specific fields"""
        effect_id = re.search(r'effect_id["\s:=]*([a-z_]+)', raw_text, re.IGNORECASE)
        category = re.search(r'category["\s:=]*([A-Z]+)', raw_text, re.IGNORECASE)
        color = re.search(r'color_rgb["\s:=]*(0x[A-Fa-f0-9]+|\d+)', raw_text, re.IGNORECASE)
        behavior = re.search(r'effect_behavior["\s:=]*(.+?)(?=\n|$)', raw_text, re.IGNORECASE)

        return {
            "effect_id": effect_id.group(1) if effect_id else "custom_effect",
            "category": category.group(1) if category else "BENEFICIAL",
            "color": color.group(1) if color else "0xe9b8b3",
            "effect_behavior": behavior.group(1) if behavior else "Applies effect every tick"
        }

    def create_food(self, item_id, display_name, ai_data):
        # 1. Texture Upload
        print(f"[INFO] Uploading texture for {display_name}...")
        tex_path = f"{self.project_dir}/src/main/resources/assets/{self.mod_id}/textures/item/"
        os.makedirs(tex_path, exist_ok=True)
        uploaded = files.upload()
        if uploaded:
            os.rename(list(uploaded.keys())[0], f"{tex_path}{item_id}.png")

        # 2. JSON Resources (1.21.1 Dual-file structure)
        assets = f"{self.project_dir}/src/main/resources/assets/{self.mod_id}"
        os.makedirs(f"{assets}/models/item", exist_ok=True)
        os.makedirs(f"{assets}/items", exist_ok=True)
        os.makedirs(f"{assets}/lang", exist_ok=True)

        with open(f"{assets}/models/item/{item_id}.json", 'w') as f:
            json.dump({"parent": "minecraft:item/generated", "textures": {"layer0": f"{self.mod_id}:item/{item_id}"}}, f, indent=2)
        with open(f"{assets}/items/{item_id}.json", 'w') as f:
            json.dump({"model": {"type": "minecraft:model", "model": f"{self.mod_id}:item/{item_id}"}}, f, indent=2)

        lang_file = f"{assets}/lang/en_us.json"
        lang_data = {}
        if os.path.exists(lang_file):
            with open(lang_file, 'r') as f: lang_data = json.load(f)
        lang_data[f"item.{self.mod_id}.{item_id}"] = display_name
        with open(lang_file, 'w') as f: json.dump(lang_data, f, indent=2)

        # 3. Crafting Recipe
        data_dir = f"{self.project_dir}/src/main/resources/data/{self.mod_id}/recipe"
        os.makedirs(data_dir, exist_ok=True)
        recipe_data = {
            "type": "minecraft:crafting_shapeless",
            "category": "misc",
            "ingredients": ai_data.get('recipe_ingredients', ['minecraft:apple']),
            "result": {
                "id": f"{self.mod_id}:{item_id}",
                "count": ai_data.get('recipe_count', 1)
            }
        }
        with open(f"{data_dir}/{item_id}.json", 'w') as f:
            json.dump(recipe_data, f, indent=2)

        # Display recipe in a user-friendly format
        ingredients = ai_data.get('recipe_ingredients', [])
        ingredient_names = [ing.replace('minecraft:', '').replace('_', ' ').title() for ing in ingredients]

        print(f"\n{'='*50}")
        print(f"CRAFTING RECIPE: {display_name.upper()}")
        print(f"{'='*50}")
        print(f"Type: Shapeless (any arrangement)")
        print(f"\nIngredients required:")
        for i, (full_name, display) in enumerate(zip(ingredients, ingredient_names), 1):
            print(f"  {i}. {display} ({full_name})")
        result_count = ai_data.get('recipe_count', 1)
        print(f"\nResult: {result_count}x {display_name}")

        # Show all effects if present
        effects = ai_data.get('effects', [])
        if effects:
            print(f"\nEffects applied:")
            for effect_data in effects:
                duration_sec = effect_data['duration'] // 20
                effect_name = effect_data['effect_id'].replace('MobEffects.', '').replace('_', ' ').title()
                print(f"   - {effect_name} for {duration_sec} seconds")

        print(f"{'='*50}\n")

        # 4. Generate Java code with optional effects
        java_dir = f"{self.project_dir}/src/main/java/{self.package_path}"
        package_name = self.package_path.replace('/', '.')

        if ai_data.get('has_effect', False) and ai_data.get('effects'):
            # Build effect instances
            effect_instances = []
            for effect_data in ai_data['effects']:
                effect_instances.append(
                    f"new MobEffectInstance({effect_data['effect_id']}, {effect_data['duration']}, {ai_data['amplifier']})"
                )

            # Build effect application code
            effects_code = ""
            for i, instance in enumerate(effect_instances):
                effects_code += f"                .onConsume(new ApplyStatusEffectsConsumeEffect({instance}, 1.0f))\n"

            # With effects
            mod_items_code = f"""package {package_name};

import net.fabricmc.fabric.api.itemgroup.v1.ItemGroupEvents;
import net.minecraft.core.Registry;
import net.minecraft.core.registries.BuiltInRegistries;
import net.minecraft.core.registries.Registries;
import net.minecraft.resources.Identifier;
import net.minecraft.resources.ResourceKey;
import net.minecraft.world.effect.MobEffectInstance;
import net.minecraft.world.effect.MobEffects;
import net.minecraft.world.food.FoodProperties;
import net.minecraft.world.item.CreativeModeTabs;
import net.minecraft.world.item.Item;
import net.minecraft.world.item.component.Consumable;
import net.minecraft.world.item.component.Consumables;
import net.minecraft.world.item.consume_effects.ApplyStatusEffectsConsumeEffect;
import java.util.function.Function;

public class ModItems {{
    public static final FoodProperties {item_id.upper()}_FOOD = new FoodProperties.Builder()
            .nutrition({ai_data['nutrition']})
            .saturationModifier({ai_data['saturation']}f)
            .alwaysEdible().build();

    public static final Consumable {item_id.upper()}_CONSUMABLE = Consumables.defaultFood()
{effects_code}            .build();

    public static final Item {item_id.upper()} = register("{item_id}", Item::new,
            new Item.Properties().food({item_id.upper()}_FOOD, {item_id.upper()}_CONSUMABLE));

    public static <GenericItem extends Item> GenericItem register(String name, Function<Item.Properties, GenericItem> itemFactory, Item.Properties settings) {{
        ResourceKey<Item> itemKey = ResourceKey.create(Registries.ITEM, Identifier.fromNamespaceAndPath({self.main_class_name}.MOD_ID, name));
        GenericItem item = itemFactory.apply(settings.setId(itemKey));
        Registry.register(BuiltInRegistries.ITEM, itemKey, item);
        return item;
    }}

    public static void initialize() {{
        ItemGroupEvents.modifyEntriesEvent(CreativeModeTabs.FOOD_AND_DRINKS).register(entries -> {{
            entries.accept({item_id.upper()});
        }});
    }}
}}
"""
        else:
            # No effect version (simpler)
            mod_items_code = f"""package {package_name};

import net.fabricmc.fabric.api.itemgroup.v1.ItemGroupEvents;
import net.minecraft.core.Registry;
import net.minecraft.core.registries.BuiltInRegistries;
import net.minecraft.core.registries.Registries;
import net.minecraft.resources.Identifier;
import net.minecraft.resources.ResourceKey;
import net.minecraft.world.food.FoodProperties;
import net.minecraft.world.item.CreativeModeTabs;
import net.minecraft.world.item.Item;
import java.util.function.Function;

public class ModItems {{
    public static final FoodProperties {item_id.upper()}_FOOD = new FoodProperties.Builder()
            .nutrition({ai_data['nutrition']})
            .saturationModifier({ai_data['saturation']}f)
            .build();

    public static final Item {item_id.upper()} = register("{item_id}", Item::new,
            new Item.Properties().food({item_id.upper()}_FOOD));

    public static <GenericItem extends Item> GenericItem register(String name, Function<Item.Properties, GenericItem> itemFactory, Item.Properties settings) {{
        ResourceKey<Item> itemKey = ResourceKey.create(Registries.ITEM, Identifier.fromNamespaceAndPath({self.main_class_name}.MOD_ID, name));
        GenericItem item = itemFactory.apply(settings.setId(itemKey));
        Registry.register(BuiltInRegistries.ITEM, itemKey, item);
        return item;
    }}

    public static void initialize() {{
        ItemGroupEvents.modifyEntriesEvent(CreativeModeTabs.FOOD_AND_DRINKS).register(entries -> {{
            entries.accept({item_id.upper()});
        }});
    }}
}}
"""

        with open(os.path.join(java_dir, "ModItems.java"), 'w') as f:
            f.write(mod_items_code)

        # Inject initialize() hook
        main_file = os.path.join(java_dir, f"{self.main_class_name}.java")
        with open(main_file, 'r') as f: content = f.read()
        if "ModItems.initialize();" not in content:
            content = re.sub(r'(public void onInitialize\(\) \{)', r'\1\n        ModItems.initialize();', content)
            with open(main_file, 'w') as f: f.write(content)

    def create_potion(self, potion_id, display_name, ai_data):
        """Create a custom potion with brewing recipe"""
        print(f"[INFO] Creating potion: {display_name}...")

        # 1. Register potion in ModPotions class
        java_dir = f"{self.project_dir}/src/main/java/{self.package_path}"
        package_name = self.package_path.replace('/', '.')

        # Parse effects
        effect_instances = []
        for effect_data in ai_data.get('effects', []):
            duration = effect_data['duration']
            effect_id = effect_data['effect_id']
            effect_instances.append(
                f"new MobEffectInstance({effect_id}, {duration}, 0)"
            )

        effects_str = ", ".join(effect_instances) if effect_instances else "new MobEffectInstance(MobEffects.SPEED, 100, 0)"

        # ModPotions.java
        mod_potions_code = f"""package {package_name};

import net.minecraft.core.Registry;
import net.minecraft.core.registries.BuiltInRegistries;
import net.minecraft.resources.Identifier;
import net.minecraft.world.effect.MobEffectInstance;
import net.minecraft.world.effect.MobEffects;
import net.minecraft.world.potion.Potion;

public class ModPotions {{
    public static final Potion {potion_id.upper()}_POTION = Registry.register(
            BuiltInRegistries.POTION,
            Identifier.fromNamespaceAndPath({self.main_class_name}.MOD_ID, "{potion_id}"),
            new Potion("{potion_id}", {effects_str})
    );

    public static void initialize() {{
        // Potions are registered on class initialization
    }}
}}
"""

        with open(os.path.join(java_dir, "ModPotions.java"), 'w') as f:
            f.write(mod_potions_code)

        # 2. Create brewing recipe registry
        brewing_code = f"""package {package_name};

import net.fabricmc.fabric.api.recipe.v1.FabricBrewingRecipeRegistryBuilder;
import net.minecraft.core.registries.BuiltInRegistries;
import net.minecraft.world.item.Items;
import net.minecraft.world.potion.Potions;

public class ModBrewingRecipes {{
    public static void registerRecipes() {{
        FabricBrewingRecipeRegistryBuilder.BUILD.register(builder -> {{
            builder.addMix(
                    Potions.WATER,
                    Items.{self._item_to_constant(ai_data.get('brewing_ingredient', 'minecraft:sugar'))},
                    BuiltInRegistries.POTION.wrapAsHolder(ModPotions.{potion_id.upper()}_POTION)
            );
        }});
    }}
}}
"""

        with open(os.path.join(java_dir, "ModBrewingRecipes.java"), 'w') as f:
            f.write(brewing_code)

        # 3. Add translation
        assets = f"{self.project_dir}/src/main/resources/assets/{self.mod_id}"
        lang_file = f"{assets}/lang/en_us.json"
        lang_data = {}
        if os.path.exists(lang_file):
            with open(lang_file, 'r') as f: lang_data = json.load(f)
        lang_data[f"item.minecraft.potion.effect.{potion_id}"] = display_name
        with open(lang_file, 'w') as f: json.dump(lang_data, f, indent=2)

        # Display potion info
        print(f"\n{'='*50}")
        print(f"POTION: {display_name.upper()}")
        print(f"{'='*50}")
        print(f"Potion ID: {potion_id}")
        print(f"Brewing ingredient: {ai_data.get('brewing_ingredient', 'minecraft:sugar')}")
        print(f"Input potion: {ai_data.get('input_potion', 'water')}")
        print(f"\nEffects applied:")
        for effect_data in ai_data.get('effects', []):
            duration_sec = effect_data['duration'] // 20
            effect_name = effect_data['effect_id'].replace('MobEffects.', '').replace('_', ' ').title()
            print(f"   - {effect_name} for {duration_sec} seconds")
        print(f"{'='*50}\n")

    def create_custom_effect(self, effect_id, display_name, ai_data):
        """Create a custom mob effect class"""
        print(f"[INFO] Creating custom effect: {display_name}...")

        java_dir = f"{self.project_dir}/src/main/java/{self.package_path}"
        package_name = self.package_path.replace('/', '.')

        effect_class_name = "".join(word.capitalize() for word in effect_id.split('_')) + "Effect"

        # Parse category
        category = ai_data.get('category', 'BENEFICIAL')
        color = ai_data.get('color', '0xe9b8b3')
        behavior = ai_data.get('effect_behavior', 'Applies effect every tick')

        # Generate effect class
        effect_class_code = f"""package {package_name};

import net.minecraft.world.effect.MobEffect;
import net.minecraft.world.effect.MobEffectCategory;
import net.minecraft.world.entity.LivingEntity;
import net.minecraft.server.level.ServerLevel;
import net.minecraft.world.entity.player.Player;

public class {effect_class_name} extends MobEffect {{
    protected {effect_class_name}() {{
        super(MobEffectCategory.{category}, {color});
    }}

    @Override
    public boolean shouldApplyEffectTickThisTick(int duration, int amplifier) {{
        return true; // Apply every tick
    }}

    @Override
    public boolean applyEffectTick(ServerLevel world, LivingEntity entity, int amplifier) {{
        // Effect behavior: {behavior}
        if (entity instanceof Player) {{
            Player player = (Player) entity;
            // TODO: Implement custom effect logic here
            // Amplifier level: amplifier (0-4)
        }}
        return super.applyEffectTick(world, entity, amplifier);
    }}
}}
"""

        with open(os.path.join(java_dir, f"{effect_class_name}.java"), 'w') as f:
            f.write(effect_class_code)

        # Create ModEffects registration class
        mod_effects_code = f"""package {package_name};

import net.minecraft.core.Registry;
import net.minecraft.core.registries.BuiltInRegistries;
import net.minecraft.core.Holder;
import net.minecraft.resources.Identifier;
import net.minecraft.world.effect.MobEffect;

public class ModEffects {{
    public static final Holder<MobEffect> {effect_id.upper()} =
            Registry.registerForHolder(
                    BuiltInRegistries.MOB_EFFECT,
                    Identifier.fromNamespaceAndPath({self.main_class_name}.MOD_ID, "{effect_id}"),
                    new {effect_class_name}()
            );

    public static void initialize() {{
        // Effects are registered on class initialization
    }}
}}
"""

        with open(os.path.join(java_dir, "ModEffects.java"), 'w') as f:
            f.write(mod_effects_code)

        # Add texture placeholder (18x18)
        tex_path = f"{self.project_dir}/src/main/resources/assets/{self.mod_id}/textures/mob_effect/"
        os.makedirs(tex_path, exist_ok=True)
        print(f"[INFO] Add texture at: {tex_path}{effect_id}.png (18x18 PNG)")

        # Add translation
        assets = f"{self.project_dir}/src/main/resources/assets/{self.mod_id}"
        lang_file = f"{assets}/lang/en_us.json"
        lang_data = {}
        if os.path.exists(lang_file):
            with open(lang_file, 'r') as f: lang_data = json.load(f)
        lang_data[f"effect.{self.mod_id}.{effect_id}"] = display_name
        with open(lang_file, 'w') as f: json.dump(lang_data, f, indent=2)

        print(f"\n{'='*50}")
        print(f"CUSTOM EFFECT: {display_name.upper()}")
        print(f"{'='*50}")
        print(f"Effect ID: {effect_id}")
        print(f"Category: {category}")
        print(f"Color: {color}")
        print(f"Behavior: {behavior}")
        print(f"Class name: {effect_class_name}")
        print(f"{'='*50}\n")

    def _item_to_constant(self, minecraft_item):
        """Convert minecraft:item_name to Items.ITEM_NAME"""
        item_name = minecraft_item.replace('minecraft:', '').upper()
        return item_name

    def finish(self):
        shutil.make_archive("/content/balanced_mod", 'zip', self.project_dir)
        print("[SUCCESS] Mod created and ready. Downloading...")
        files.download("/content/balanced_mod.zip")

agent = BalancedItemAgent()

In [ ]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"qwen2.5:0.5b","model":"qwen2.5:0.5b","modified_at":"2026-01-01T22:14:52.646653142Z","size":397821319,"digest":"a8b0c51577010a279d933d14c2a8ab4b268079d44c5c8830c0a93900f1827c67","details":{"parent_model":"","format":"gguf","family":"qwen2","families":["qwen2"],"parameter_size":"494.03M","quantization_level":"Q4_K_M"}},{"name":"qwen2.5-coder:1.5b","model":"qwen2.5-coder:1.5b","modified_at":"2026-01-01T21:44:49.575150478Z","size":986062089,"digest":"d7372fd828518a4d38b1eb196c673c31a85f2ed302b3d1e406c4c2d1b64a0668","details":{"parent_model":"","format":"gguf","family":"qwen2","families":["qwen2"],"parameter_size":"1.5B","quantization_level":"Q4_K_M"}}]}

In [ ]:
# 1. Start the process
uploaded = files.upload()
agent.setup_project(list(uploaded.keys())[0])

# 4. Test Custom Effect Example
print("\n" + "="*60)
print("TESTING CUSTOM EFFECT")
print("="*60)
description3 = "A custom effect that make u explode in 5 seconds"
effect_id = "explodio"
effect_name = "explodio"

logic3 = agent.get_balanced_logic(description3)
print(f"\n[DEBUG] Extracted logic:\n{logic3}\n")

if logic3["type"] == "EFFECT":
    agent.create_custom_effect(effect_id, effect_name, logic3)

print("\n[SUCCESS] All items created successfully!")
agent.finish()

Saving taco-template-1.21.11.zip to taco-template-1.21.11.zip
✅ Loaded Mod: taco | Main: Taco

✨ TESTING CUSTOM EFFECT
⚠️ Parser error: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79851d4a4e90>: Failed to establish a new connection: [Errno 111] Connection refused'))

📊 Extracted Logic:
{'type': 'FOOD', 'nutrition': 6, 'saturation': 0.6, 'effects': [], 'recipe_ingredients': ['minecraft:apple', 'minecraft:sugar'], 'recipe_count': 1}


✅ All items created successfully!
🎁 SUCCESS! Your mod is balanced and ready with crafting recipes.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import time
import requests

print("Testing Ollama response time...")
print("=" * 50)

# Quick test
start = time.time()
try:
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "qwen2.5:0.5b",
            "prompt": "Say 'OK' only",
            "stream": False
        },
        timeout=30
    )
    elapsed = time.time() - start
    print(f"[SUCCESS] Response received in {elapsed:.1f} seconds")
    print(f"Response: {response.json().get('response', '')[:100]}")
except requests.exceptions.Timeout:
    elapsed = time.time() - start
    print(f"[ERROR] TIMEOUT after {elapsed:.1f} seconds")
    print("Model is too slow even for simple requests")
except Exception as e:
    elapsed = time.time() - start
    print(f"[ERROR] Failed after {elapsed:.1f} seconds: {e}")

🔍 Testing Ollama Response Time...
✅ Response received in 3.0 seconds
Response: OK! How can I assist you today?


In [ ]:
# Install Java 21
!apt-get install openjdk-21-jdk-headless -qq > /dev/null

# Set the environment variables so Gradle uses Java 21
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
!java -version

openjdk version "21.0.9" 2025-10-21
OpenJDK Runtime Environment (build 21.0.9+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 21.0.9+10-Ubuntu-122.04, mixed mode, sharing)


In [ ]:
import os
import subprocess

# 1. Ensure the Gradlew script is executable
gradle_path = os.path.join(agent.project_dir, "gradlew")
!chmod +x {gradle_path}

# 2. Run the Build
print("[INFO] Building mod... This may take 1-2 minutes for the first run.")
try:
    # We use --no-daemon for Colab to save memory
    result = subprocess.run(
        [gradle_path, "build"],
        cwd=agent.project_dir,
        capture_output=True,
        text=True
    )

    if result.returncode == 0:
        print("[SUCCESS] Build completed successfully!")

        # 3. Locate the generated .jar
        libs_dir = os.path.join(agent.project_dir, "build/libs")
        if os.path.exists(libs_dir):
            jars = [f for f in os.listdir(libs_dir) if f.endswith(".jar") and "-dev" not in f and "-sources" not in f]
            if jars:
                final_jar = os.path.join(libs_dir, jars[0])
                print(f"[INFO] Found mod file: {jars[0]}")
                files.download(final_jar)
            else:
                print("[ERROR] Could not find the final .jar file in build/libs.")
    else:
        print("[ERROR] Build failed! Check the errors below:")
        print(result.stdout)
        print(result.stderr)

except Exception as e:
    print(f"[ERROR] An error occurred during the build process: {e}")

🔨 Building Mod... This may take 1-2 minutes for the first run.
✅ Build Successful!
📦 Found Mod File: juice-1.0.0.jar


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>